In [36]:
import numpy as np
from random import uniform
from sklearn.preprocessing import normalize

In [57]:
def rand_matrix(shape):
    res = np.zeros(shape=shape)
    min_value = 0.01
    max_value = 1
    for i in range(shape[1]):
        col_sum = 0
        for j in range(shape[0]):
            if j == shape[0] - 1:
                res[j][i] = 1.0 - col_sum
            else:
                res[j][i] = uniform(min_value, max_value) / shape[0]
                col_sum += res[j][i]
    return res

In [35]:
dictionary = ['w0', 'w1', 'w2']

data = [
    ['w0', 'w1', 'w1'],
    ['w0', 'w1', 'w2'],
    ['w0', 'w2', 'w2']
]

In [81]:
def em_step(wt, td, data, dictionary, cl=2):
    fi = np.zeros(shape=(len(data), cl))
    theta = np.zeros(shape=(cl, len(dictionary)))
    
    def n_w_d(word, doc):
        return data[doc].count(dictionary[word])
    
    def p_w_d(word, doc):
        s = 0
        for i in range(cl):
            s += wt[word][i] * td[i][doc]
        return s
    
    def p_t_w_d(c, word, doc):
        return wt[word][c] * td[c][doc] / p_w_d(word, doc)
    
    def n_d_w_t(c, word, doc):
        return n_w_d(word, doc) * p_t_w_d(c, word, doc)
    
    def n_w_t(c, word):
        s = 0
        for i in range(len(data)):
            s += n_d_w_t(c, word, i)
        return s
    
    def n_t_d(c, doc):
        s = 0
        for i in range(len(data[doc])):
            s += n_d_w_t(c, i, doc)
        return s
    
    def n_t(c):
        s = 0
        for i in range(len(dictionary)):
            s += n_w_t(c, i)
        return s
    
    def n_d(doc):
        s = 0
        for i in range(cl):
            s += n_t_d(i, doc)
        return s
    
    
    for i in range(fi.shape[0]):
        for j in range(fi.shape[1]):
            fi[i][j] = n_w_t(j, i) / n_t(j)
            
    for i in range(theta.shape[0]):
        for j in range(theta.shape[1]):
            theta[i][j] = n_t_d(i, j) / n_d(j)
            
    return fi, theta            

In [90]:
def EM(data, dictionary, cl=2, iter_num=5):
    word_t = normalize(rand_matrix([3, 2]), axis=0)
    t_doc = normalize(rand_matrix([2, 3]), axis=0)
    
    word_matrix = word_t
    doc_matrix = t_doc
    
    for it in range(iter_num):
        prob_word, prob_doc = em_step(word_matrix, doc_matrix, data, dictionary, 2)
        word_matrix = prob_word
        doc_matrix = prob_doc
        
        print("word_matrix\n", word_matrix)
        print("doc_matrix\n", doc_matrix)
        print("-"*40)

In [93]:
EM(data, dictionary, iter_num=5)

word_matrix
 [[0.49287201 0.31246472]
 [0.16515024 0.35533269]
 [0.34197775 0.33220259]]
doc_matrix
 [[0.13255514 0.15222337 0.06224652]
 [0.86744486 0.84777663 0.93775348]]
----------------------------------------
word_matrix
 [[0.50804332 0.31141361]
 [0.20896328 0.34893724]
 [0.2829934  0.33964916]]
doc_matrix
 [[0.10895005 0.15124773 0.07423379]
 [0.89104995 0.84875227 0.92576621]]
----------------------------------------
word_matrix
 [[0.50995766 0.31138842]
 [0.23413638 0.34565818]
 [0.25590595 0.3429534 ]]
doc_matrix
 [[0.10091883 0.15031444 0.08031221]
 [0.89908117 0.84968556 0.91968779]]
----------------------------------------
word_matrix
 [[0.50894326 0.31157153]
 [0.25024746 0.34362944]
 [0.24080928 0.34479903]]
doc_matrix
 [[0.09886653 0.1494181  0.08248929]
 [0.90113347 0.8505819  0.91751071]]
----------------------------------------
word_matrix
 [[0.50713964 0.31180704]
 [0.26354643 0.34197659]
 [0.22931392 0.34621637]]
doc_matrix
 [[0.09998394 0.14855454 0.08207135]
 [0

После пяти итераций: все документы с достоточно большой вероятностью принадлежат ко второй теме; в первой теме наиболее вероятно слово w0, ко второй все слова практически равовероятны (слово w2 с небольшим перевесом побеждает)
